In [1]:
#options
metCut = 1
cutLogg = True
norm = True




from metaimports import *
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from modelValidator import psMetric, cut, shuffleDwarfs, plotRes, plot, norm_inputs
from sklearn.neural_network import MLPRegressor
from matplotlib.pyplot import figure
fits = fitsParser(includefile = 'FeHdata.txt')
cut(fits)

if cutLogg:
    #dwarfs = fits.data
    dwarfs = (fits.data.sample(frac = 1, random_state = int(1000*np.random.rand())))
    dwarfs = dwarfs[dwarfs['logg']>3.5]
else:
    dwarfs = shuffleDwarfs(fits)

metPoorDwarfs = dwarfs[dwarfs['FeHadop']<metCut]
metRichDwarfs = dwarfs[dwarfs['FeHadop']>metCut]
din = metPoorDwarfs[fits.colours]
if norm:
    normParams = norm_inputs(din)
dout = metPoorDwarfs[fits.outputs]
din_train, din_test, dout_train, dout_test = train_test_split(din,dout, test_size = 0.7, shuffle = False)
#din_train = din_train.sample(frac = 1, random_state = int(1000*np.random.rand()))

Using TensorFlow backend.



Data saved

condition on dg_fpsf cut 22 inputs
condition on dr_fpsf cut 23 inputs
condition on di_fpsf cut 6 inputs
condition on dz_fpsf cut 2 inputs
condition on dy_fpsf cut 14 inputs
condition on dCaHK_0 cut 14 inputs
condition on du_0 cut 158 inputs
condition on dFeHadop cut 280 inputs
10.66% of inputs cut


In [2]:
kernel1 = RBF(length_scale = 0.5)
kernel2 = WhiteKernel(noise_level = 0.1)
kernel = kernel1 + kernel2


params = {
    'kernel' : kernel,
    'n_restarts_optimizer' : 0,
    'normalize_y' : True,
    'copy_X_train' : False,
    'alpha' : 1e-4
}



model = gpr(**params)
model.fit(din_train, dout_train)

KeyboardInterrupt: 

In [ ]:
getbins = lambda data, size = 0.1 : np.arange(min(data), max(data) + size, size)
metPoorDwarfs['test_flag'] = np.append(np.zeros(len(din_train)),np.ones(len(din_test)))
metRichDwarfs['test_flag'] = [-1]*len(metRichDwarfs)
output = metPoorDwarfs.append(metRichDwarfs).sort_index()
check = filter(lambda x : x != 'test_flag' , output.keys())
#assert (output[check].sort_index() == dwarfs).all().all()
inp = output[fits.colours]
if norm:
    for field in fits.colours:
        inp[field] = (inp[field] - normParams[field][0])*1.0/normParams[field][1]
        
output['FeH_pred'] = model.predict(inp)
output['res'] = output['FeH_pred'] - output['FeHadop']
output = output.reset_index()
x = plotRes((output['res'])[output['test_flag']==1], (output['res'])[output['test_flag']==0], metLabel = '[Fe/H]<%.2f'%metCut, size = 0.05,
       xy = (257, 195) , normalFit  = False)
toPlot = (output[output['FeHadop']<metCut])['FeHadop']
toPlot -= np.mean(toPlot)
toPlot *= -1
plt.hist(toPlot, histtype = 'step' , bins = getbins(toPlot, size = 0.05), density = True, label = 'mean(exp) - exp')
plt.legend()
plt.show()


plot((output['res'])[output['test_flag']==-1])

plt.scatter(output['FeHadop'], output['FeH_pred'] , s = 0.5)
plt.title('Predicted v.s. Expected [Fe/H]')
plt.xlabel('[Fe/H] (Expected)')
plt.ylabel('[Fe/H] (Predicted)')
x = np.linspace(max((plt.xlim())[0], (plt.ylim())[0]), min((plt.xlim())[1], (plt.ylim())[1]))
plt.plot(x,x, color = 'black')
plt.show()

plt.scatter((output[output['FeHadop']<metCut])['FeHadop'], (output[output['FeHadop']<metCut])['FeH_pred'] , s = 0.5)
plt.title('Predicted v.s. Expected [Fe/H]')
plt.xlabel('[Fe/H] (Expected)')
plt.ylabel('[Fe/H] (Predicted)')
x = np.linspace(max((plt.xlim())[0], (plt.ylim())[0]), min((plt.xlim())[1], (plt.ylim())[1]))
plt.plot(x,x, color = 'black')
plt.show()


getbins = lambda data, size = 0.1 : np.arange(min(data), max(data) + size, size)
den = False
getNbins = lambda data, s ,size = 0.1 : int((np.max(data[s])-np.min(data[s]))/size)
try:
    plt.hist((output['FeH_pred'])[output['test_flag']==-1], bins  = getbins((output['FeH_pred'])[output['test_flag']==-1]),  histtype = 'step', label = 'Predicted | [Fe/H]>%.1f'%metCut, density = den)
except ValueError:
    pass
plt.hist((output['FeHadop']), bins = getbins((output['FeHadop'])), histtype = 'step', label = 'Expected', density = den)
plt.hist((output['FeH_pred'])[output['test_flag']>=0], bins=getbins((output['FeH_pred'])[output['test_flag']>=0]),  histtype = 'step', label = 'Predicted | [Fe/H]<%.1f'%metCut, density = den)
plt.xlabel('[Fe/H]')
plt.title('Distributions of Predicted and Expected Metallicities', fontsize = 15)
plt.legend(loc = 'upper right')
plt.show()


bins = [0, -0.5, -1,-1.5, -2 , -3]
f,  z = plt.subplots(1, len(bins), sharex='all', sharey='row')
plt.xlim(-2,2)
f.set_size_inches(18.5, 10.5)
for i in range(len(bins)):
    if (i==len(bins)-1):
        sub = output[output['FeHadop']<bins[i]]
    else:
        sub = output[(output['FeHadop']<bins[i])&(output['FeHadop']>bins[i+1])]
    Title = '%.1f<[Fe/H]<%.1f' %(bins[i+1] ,bins[i]) if i+1 in range(len(bins)) else '[Fe/H]<%.1f'%bins[i]
    plotRes((sub[sub['test_flag']!=0])['res'], (sub[sub['test_flag']==0]['res']), toPlot = z[i],title = Title, normalFit = False, xy = (0.1 , 0.9))
z[0].legend(loc = 'upper left')
f.text(0.5, 0.08, '[Fe/H] predicted - expected', ha='center', fontsize = 15)
f.text(0.09, 0.5, 'Probability Density', va='center', rotation='vertical', fontsize = 15)
plt.show()

In [ ]:
inc = [-1, -1.5 , -2, -2.5, -3]
pur , com = psMetric(output, inc = inc)
def printTable(inc, pur, com):
    table = ""
    table += '| [Fe/H]< | purity | completness |'
    table += '\n'+'-'*len(table)+'\n'
    for i in range(len(inc)):
        if pur[i]>=0:
            table+= '| %-8.1f| %6.4f | %-11.4f |\n'%(inc[i],pur[i],com[i])
        else: table+= '| %-8.1f| %5.3f | %-11.4f |\n'%(inc[i],pur[i],com[i])
    return table
            
print printTable(inc,pur,com)
#joblib.dump(model, 'min%scutnn'% str(-1*metCut))
#save_data(output, 'min2cut')